<a href="https://colab.research.google.com/github/SakaethRam/ZREX-Wingman/blob/main/Wingman%20Beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-------------------------------------------------------------------------- PIP INSTALLS -----------------------------------------------------------------------------
!pip install SpeechRecognition openai googletrans==4.0.0-rc1 gtts pillow

In [ ]:
import speech_recognition as sr
import openai
from googletrans import Translator
from gtts import gTTS
import os
import tkinter as tk
from PIL import Image, ImageTk

#----------------------------------------------------- Function to recognize speech and convert it to text -------------------------------------------------------------
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    try:
        text = recognizer.recognize_google(audio, language="auto")
        print(f"User said: {text}")
        return text
    except sr.UnknownValueError:
        print("Sorry, could not understand the audio.")
        return None
    except sr.RequestError:
        print("Could not request results, please check your connection.")
        return None


In [ ]:
#--------------------------------------------------------- Function to translate text to a target language -----------------------------------------------------------
def translate_text(text, target_lang="en"):
    translator = Translator()
    translation = translator.translate(text, dest=target_lang)
    return translation.text, translation.src

#---------------------------------------------------------- Function to get response from OpenAI's GPT model -----------------------------------------------------------
def get_gpt_response(prompt):
    openai.api_key = "your-openai-api-key"
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response["choices"][0]["message"]["content"]

#-------------------------------------------------------- Function to convert text to speech and play the audio ---------------------------------------------------------
def speak_text(text, lang="en"):
    tts = gTTS(text=text, lang=lang)
    tts.save("response.mp3")
    os.system("start response.mp3" if os.name == "nt" else "afplay response.mp3")

#------------------------------------------------------------- Function to start/stop voice assistant ---------------------------------------------------------------------
def toggle_voice_assistant():
    global listening
    if listening:
        listening = False
        status_label.config(text="Press to Start Listening")
    else:
        listening = True
        status_label.config(text="Listening...")
        user_input = recognize_speech()
        if user_input:
            translated_input, detected_lang = translate_text(user_input, "en")
            if detected_lang != "en":
                print(f"Translated Prompt (English): {translated_input}")
            gpt_response = get_gpt_response(translated_input)
            print(f"GPT Response ({detected_lang}): {gpt_response}")
            speak_text(gpt_response, detected_lang)
            print("Do you need the response in English subtitles? (yes/no)")
            user_confirmation = recognize_speech()
            if user_confirmation and "yes" in user_confirmation.lower():
                english_subtitle, _ = translate_text(gpt_response, "en")
                print(f"English Subtitle: {english_subtitle}")

In [ ]:
#------------------------------------------------------------------------- IMAGE UPLOAD -----------------------------------------------------------------------------
from google.colab import files

# Upload image
uploaded = files.upload()

# Get the file path of the uploaded image
image_path = list(uploaded.keys())[0]  # Gets the uploaded file name dynamically

In [ ]:
#-------------------------------------------------------------------------- GUI Setup -----------------------------------------------------------------------------
root = tk.Tk()
root.title("Voice Assistant")
root.geometry("300x400")

# Load button image
image_path = "button.png"  # Change this to the path of your chosen PNG file
button_image = Image.open(image_path)
button_image = button_image.resize((200, 200))
button_photo = ImageTk.PhotoImage(button_image)

# Button to start/stop voice assistant
listening = False
status_label = tk.Label(root, text="Press to Start Listening", font=("Arial", 12))
status_label.pack(pady=10)
voice_button = tk.Button(root, image=button_photo, command=toggle_voice_assistant, borderwidth=0)
voice_button.pack()

# Run GUI
root.mainloop()